In [ ]:
########################################################################
# Author    : A. Alsema, J. Kotah
# Date      : October 2023
# Dataset   : Visium Spatial Transcriptomics for MS lesions, 14 WM samples
# Purpose   : define hub genes from SpaceX analysis
# Output    : list of hubs per sample, networks per sample
# Input     : RDS SpaceX analyzed object per sample
########################################################################


In [3]:
rm(list = ls())
library(tidyverse)
library(corrr)
library(igraph)
library(patchwork)


In [2]:
outdir = "YOUR OUTPUT DIRECTORY HERE" 

#load sample analyzed
ID <- "ST33"

path = "POST-PROCESSED FILE OUTPUT FOLDER"
path_to_load = paste0(path, ID, "_003_SpaceX_Post-processed.rds")
sample = readRDS(path_to_load)

#load sheet created in Script 2 about order of spatial clusters analyzed, check which folder
cluster_info_dir = "./Order_of_groups/"
cluster_info_df = read.csv(paste0(cluster_info_dir, ID, "_orderofgroups.csv"))
cluster_info_df

X,x
<int>,<chr>
1,ST33_active
2,ST33_PL_active
3,ST33_RIM_active


In [6]:
#preparing to load all samples in a loop
samples = c("ST31",
            "ST32",
            "ST33",
            "ST34",
            "ST37",
            "ST67",
            "ST68",
            "ST69",
            "ST70",
            "ST71",
            "ST72",
            "ST73",
            "ST74",
            "ST79")

length(samples)

[1] 14

In [7]:
#create histograms to visualize all correlation values that SpaceX gives
#from this we chose to use a value of 0.7 as a cutoff to determine 'connection' between two genes

pdf("Correlation_Histograms.pdf")

for (ID in samples){
    print(ID)
    
    path_to_load = paste0(path, ID, "_003_SpaceX_Post-processed.rds")
    
    #loading sample
    sample = readRDS(path_to_load)
        
    #loading cluster network order within sample, here everything is analyzed
    #for our analysis specifically, the rims were always in index 3 in active and act/inact lesions
    cluster_info_df = read.csv(paste0(cluster_info_dir, ID, "_orderofgroups.csv"))

    for (each in 1:nrow(cluster_info_df)){
        cluster_index = each
        cluster_name = cluster_info_df[cluster_index, 'x']
        print(cluster_name)
        
        #select the spatial cluster specific network
        group_network <- sample[[3]][, , cluster_index] 
        above_thres = group_network[abs(group_network) > 0.7] %>% length() 

        abs(as.matrix(group_network)) %>% hist(main = paste0("Histogram of |correlation values| in ", cluster_name),
                                       sub = paste0("# Connections > +/-0.7: ", above_thres)) %>%
        
        #500 genes were loaded, so we expect to see at least 500 good connections due auto-correlation, this highlights those
        abline(h=500, col = 'red', lty = 'dashed') %>% 
        
        #we chose an absolute value of 0.7, which we visualize here
        abline(v = 0.7, col = "blue", lty = "dashed")
    }
    
    rm(sample)
    rm(cluster_info_df)
}

dev.off()

[1] "ST31"
[1] "ST31_CWM"
[1] "ST32"
[1] "ST32_CWM"
[1] "ST33"
[1] "ST33_active"
[1] "ST33_PL_active"
[1] "ST33_RIM_active"
[1] "ST34"
[1] "ST34_NAWM"
[1] "ST37"
[1] "ST37_NAWM"
[1] "ST67"
[1] "ST67_chronic"
[1] "ST67_PL_chronic"
[1] "ST67_RIM_chronic"
[1] "ST68"
[1] "ST68_chronic"
[1] "ST68_PL_chronic"
[1] "ST68_RIM_chronic"
[1] "ST69"
[1] "ST69_active"
[1] "ST69_PL_active"
[1] "ST69_RIM_active"
[1] "ST70"
[1] "ST70_active"
[1] "ST70_PL_active"
[1] "ST70_RIM_active"
[1] "ST71"
[1] "ST71_chronic"
[1] "ST71_PL_chronic"
[1] "ST71_RIM_chronic"
[1] "ST72"
[1] "ST72_chronic"
[1] "ST72_PL_chronic"
[1] "ST72_RIM_chronic"
[1] "ST73"
[1] "ST73_chronic"
[1] "ST73_PL_chronic"
[1] "ST73_RIM_chronic"
[1] "ST74"
[1] "ST74_chronic"
[1] "ST74_PL_chronic"
[1] "ST74_RIM_chronic"
[1] "ST79"
[1] "ST79_active"
[1] "ST79_PL_active"
[1] "ST79_RIM_active"


png 
  2

In [30]:
#hub determination was done by selecting genes connected to at least 10% of total genes in the network 
#(i.e., those above +0.7)

hub.list.top10p = list()

pdf("Visualize_hub_gene_determination.pdf")
for (ID in samples){
    print(paste0("Processing sample ", ID))
    
    #load sample info and data
    
    path_to_load = paste0(path, ID, "_003_SpaceX_Post-processed.rds")
    sample = readRDS(path_to_load)    
    cluster_info_df = read.csv(paste0(cluster_info_dir, ID, "_orderofgroups.csv"))


for (cluster_index in 1:nrow(cluster_info_df)){

    #for completeness we also do other spatial clusters;
    #in our analysis, we focused on cluster_index =3, the active and act/inact lesion rims
    group_network <- sample[[3]][, , cluster_index] 
    
    # convert to matrix
    colnames(group_network) <- sample$Genes
    rownames(group_network) <- sample$Genes
    group_network = as.matrix(group_network)
    diag(group_network) = 0

    #extract only the strong correlations to improve visibility
    filter = abs(group_network) < 0.7 # remove small correlations, 0.7 was a good threshold; take only positive
    result_network = group_network
    result_network[filter] = 0

    #igraph based:
    graph2 <- graph.adjacency(result_network, mode = "undirected", weighted = TRUE)
    remove <- E(graph2)$weight < 0 #edges with links that are sub-threshold
    graph2 <- delete_edges(graph2, E(graph2)[remove])

    # Find isolated nodes (nodes with degree zero)
    isolated_nodes <- V(graph2)[degree(graph2) == 0]
    # Delete isolated nodes from the network
    graph2 <- delete_vertices(graph2, v = isolated_nodes)

    #identify total nodes left in each network
    nodes = degree(graph2) %>% data.frame() %>% nrow()
    
    degree_df =  degree(graph2) %>% sort(decreasing = T) %>% data.frame() %>% `colnames<-`("Degree") %>%
    mutate(Gene = rownames(.)) %>% mutate(Degree_pct = 100* Degree/nrow(.))
    
    #show histogram and where 10 percent cutoff would be
    degree_df %>% mutate(fill = Degree_pct >= 10) %>%
    ggplot(., aes(y= Degree_pct, x = reorder(Gene, -Degree),
                      fill = fill
                     )) + geom_bar(stat="identity") +
    geom_hline(yintercept = 10, color = "red", linetype = "dashed") +
    ylab("Percent of genes in network connected to") + theme_classic() +
    xlab("Genes ordered by connectivity")
    theme(axis.text.x = element_blank(),
          axis.ticks.x = element_blank(),
          legend.position='none', aspect.ratio = 0.48
         ) +
    scale_y_continuous(expand = c(0,0)) +
    ggtitle(title = paste0("Identification of hubs in ", cluster_info_df[cluster_index,2]))

    #export hub information per spatial cluster for downstream interpretation
    hub.list.top10p[[cluster_info_df[cluster_index,2]]] = degree(graph2) %>% sort(decreasing = T) %>% data.frame() %>% `colnames<-`("Degree") %>%
    mutate(Gene = rownames(.), Percent = Degree/nodes) %>% filter(Percent > 0.1) %>% rownames(.) %>% c(.)
   
    }
    rm(sample)
    rm(cluster_info_df)    
}

dev.off()

saveRDS(hub.list.top10p, "04_allSamples_top10p_hub_genes.rds")

[1] "Processing sample ST31"
[1] "Processing sample ST32"
[1] "Processing sample ST33"
[1] "Processing sample ST34"
[1] "Processing sample ST37"
[1] "Processing sample ST67"
[1] "Processing sample ST68"
[1] "Processing sample ST69"
[1] "Processing sample ST70"
[1] "Processing sample ST71"
[1] "Processing sample ST72"
[1] "Processing sample ST73"
[1] "Processing sample ST74"
[1] "Processing sample ST79"


png 
  2

In [ ]:
#example code for plotting networks per sample

ID = samples[1] #select sample to visualize

path_to_load = paste0(path, ID, "_003_SpaceX_Post-processed.rds")
sample = readRDS(path_to_load)

cluster_info_df = read.csv(paste0(cluster_info_dir, ID, "_orderofgroups.csv"))

cluster_index = 3 #active rims
cluster_name = cluster_info_df[cluster_index, 'x']
print(cluster_name)

group_network <- sample[[3]][, , cluster_index]

# convert to matrix, extract only the strong correlations to improve visibility
colnames(group_network) <- sample$Genes
rownames(group_network) <- sample$Genes
group_network = as.matrix(group_network)

#make df  with communities and color
each = 3
cluster_index = each
cluster_name = cluster_info_df[cluster_index, 'x']
print(cluster_name)

group_network <- sample[[3]][, , cluster_index] # group network is a cluster specific network

# convert to matrix, extract only the strong correlations to improve visibility
colnames(group_network) <- sample$Genes
rownames(group_network) <- sample$Genes
group_network = as.matrix(group_network)
diag(group_network) = 0
filter = group_network < 0.7 # remove small correlations, 0.7 was a good threshold, as estimated by local minimum in probability density in 3 test samples
result_network = group_network
result_network[filter] = 0

# Convert the adjacency matrix to an igraph object
graph <- graph.adjacency(result_network, mode = "undirected", weighted = TRUE)
remove <- E(graph)$weight == 0
graph <- delete_edges(graph, E(graph)[remove])

# Find isolated nodes (nodes with degree zero)
isolated_nodes <- V(graph)[igraph::degree(graph) == 0]
# Delete isolated nodes from the graph
graph <- delete_vertices(graph, v = isolated_nodes)

# define the connectivity of all the genes
vertex_degrees <- igraph::degree(graph)
# define node size based on degree
scaling_factor = 0.25 # adjust as needed for visibility
node_sizes <- scaling_factor * vertex_degrees
communities <- cluster_walktrap(graph)# Get the community assignments for each node
node_communities <- membership(communities)
community_colors <- rainbow(length(communities))
node_colors <- community_colors[node_communities]
layout_algorithm <- layout.fruchterman.reingold
node_positions <- layout_algorithm(graph)# Plot the graph with node positions reflecting communities

#for the legend
sizeCut = round(seq(1, max(vertex_degrees), length.out = 4),0)
    
#plot top3 hubs per community
set.seed(42)
plot(graph,
     layout = layout.fruchterman.reingold,
     vertex.label.dist = 0.8, # Adjust the distance of labels from nodes
     vertex.label.cex = 0.5,
     vertex.label.color = "black",  # Label color
     vertex.color = node_colors, #color nodes by community
     vertex.size = node_sizes,  # specify node size based on degree. size is proportional to degree
     edge.width = 5,  # Edge width
     edge.color = "gray"  # Edge color
    )

legend('bottomright', title = 'Degree', legend = unique(sizeCut), pt.cex = sizeCut/20, pt.bg = "grey",
       bty = 'n', y.intersp = 1.05,
       pch= 21)


In [4]:
sessionInfo()

R version 4.1.2 (2021-11-01)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.5 LTS

Matrix products: default
BLAS/LAPACK: /data/bcn/p283607/anaconda3/envs/spacex/lib/libopenblasp-r0.3.21.so

locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] patchwork_1.2.0 igraph_1.5.1    corrr_0.4.4     lubridate_1.9.2
 [5] forcats_1.0.0   stringr_1.5.0   dplyr_1.1.3     purrr_1.0.2    
 [9] readr_2.1.4     tidyr_1.3.0     tibble_3.2.1    ggplot2_3.4.4  
[13] tidyverse_2.0.0

loaded via a namespace (and not attached):
 [1] pillar_1.9.0     compil